# Eficiencia Julia

## Julia es rápido

Muy a menudo, técnicas de comparación o punt de referencia  (_benchmark_), son utilizadas para comparar lenguajes de programación. Estos puntos de referencia pueden dar lugar a largas discusiones, primero sobre qué se está evaluando exactamente y, en segundo lugar, qué explica las diferencias. Estas simples preguntas a veces pueden volverse más complicadas de que podemos imaginar al principio.

El propósito de los siguientes ejempos es mostrar un ejercicio de _benchmark_ sencillo y comprensible.

La versión original de este matrial nace como una maravillosa conferencia de Steven Johnson en el MIT: ["Boxes and registers"](https://github.com/stevengj/18S096/blob/master/lectures/lecture1/Boxes-and-registers.ipynb)

## Contenidos

- Definir la función `sum`
- Implementaciones y _comparativas_ de `sum`
    - Julia(built-in)
    - Julia(hand-writen)
    - C (hand-writen)
    - Python (built-in)
    - Python (numpy)
    - Python (hand-writen)
- Usando paralelismo con Julia
    - Utilización de la asociatividad de punto flotante
    - Utilización de 4 cores: built-in
    - Utilización de 4 cores: hand-writen
- Resumen de las comparativas


## `sum`: una función fácil de entender

Consideremos la función que suma elementos de un vector `sum(a)`, la cual se calcula:
$$sum(a) = \sum_{i=1}^n a_i$$ donde $n$ es el número de elementos (longitud) de $a$.

In [ ]:
a = rand(10^7)    #vector 1D de números aleatorios uniformes [0,1)

In [ ]:
sum(a)

## Comparativas entre algunos cuantos lenguajes

In [ ]:
@time sum(a)

In [ ]:
@time sum(a)

In [ ]:
@time sum(a)

La macro `@` puede devolver ciertos resultados sesgados, por tanto no es la mejor elección que hagamos. En su lugar utilizaremos el paquete `BenchmarkTools.jl` para hacer comparativas fáciles y mas precisas.

In [ ]:
using Pkg
Pkg.add("BenchmarkTools")

using BenchmarkTools

In [ ]:
@benchmark sum($a)

## 1. Julia (built-in)

Bien, ya hemos visto el uso de la función predefinida `sum()`Julia. Por supuesto, está escrita en Julia, pero ¿funcionaría si escribiéramos una implementación nosotros mismos?

In [ ]:
@which sum(a)

Guardemos estos resultados de referencia en un diccionario para que podamos comenzar a realizar un seguimiento de ellos y compararlos en el futuro.

In [ ]:
j_bench = @benchmark sum($a)

In [ ]:
j_bench.times

In [ ]:
d = Dict()
d["Julia built-in"] = minimum(j_bench.times) / 1e6
d

## 2. Julia (hand-written)

In [ ]:
function mysum(A)
    s = 0.0
    for a in A
        s += a
    end
    return s
end

In [ ]:
mysum(a)

In [ ]:
j_bench_hand = @benchmark mysum($a)

In [ ]:
d["Julia hand-written"] = minimum(j_bench_hand.times) / 1e6
d

## 3. Lenguaje C

C a menudo se considera el estándar de oro: difícil para el ser humano, agradable para la máquina. Lograr estar dentro de un factor de ~2xC a menudo es satisfactorio. No obstante, incluso dentro de C, hay muchos tipos de optimizaciones posibles de las que un desarrollador de C novato puede o no aprovechar.

El autor actual no habla C, por lo que no comprende la celda de abajo, pero está feliz de saber que puede poner código C en una sesión de Julia, compilarlo y ejecutarlo. Tengamos en cuenta que `"""` envuelve una cadena de varias líneas.

In [ ]:
using Libdl
C_code = """
    #include <stddef.h>
    double c_sum(size_t n, double *X) {
        double s = 0.0;
        for (size_t i = 0; i < n; ++i) {
            s += X[i];
        }
        return s;
    }
"""

const Clib = tempname()   # make a temporary file


# compile to a shared library by piping C_code to gcc
# (works only if you have gcc installed):

open(`gcc -fPIC -O3 -msse3 -xc -shared -o $(Clib * "." * Libdl.dlext) -`, "w") do f
    print(f, C_code)
end

# define a Julia function that calls the C function:
c_sum(X::Array{Float64}) = ccall(("c_sum", Clib), Float64, (Csize_t, Ptr{Float64}), length(X), X)

In [ ]:
c_sum(a)

In [ ]:
c_bench = @benchmark c_sum($a)

In [ ]:
d["C"] = minimum(c_bench.times) / 1e6  # in milliseconds
d

## 4. Python (built in `sum`)

El paquete `PyCall` provee una interfase en Julia para usar Python

In [ ]:
using PyCall

In [ ]:
pysum = pybuiltin("sum")

In [ ]:
pysum(a)

In [ ]:
py_bench = @benchmark $pysum($a)

In [ ]:
d["Python built.in"] = minimum(py_bench.times) / 1e6
d

## 5. Python Numpy

`Numpy` es una biblioteca en C optimizada y se llama directamente desde Python:

In [ ]:
using Pkg
Pkg.add("Conda")
using Conda

In [ ]:
Conda.add("numpy")

In [ ]:
numpy_sum = pyimport("numpy")["sum"]

In [ ]:
py_numpy_bench = @benchmark $numpy_sum($a)

In [ ]:
numpy_sum(a)

In [ ]:
d["Python numpy"] = minimum(py_numpy_bench.times) / 1e6
d

# 6. Python (hand-written)

In [ ]:
py"""
def py_sum(A):
    s = 0.0
    for a in A:
        s += a
    return s
"""

sum_py = py"py_sum"

In [ ]:
py_hand = @benchmark $sum_py($a)

In [ ]:
sum_py(a)

In [ ]:
d["Python hand-written"] = minimum(py_hand.times) / 1e6
d

## Resumen

In [ ]:
for (key, value) in sort(collect(d), by=last)
    println(rpad(key, 25, "."), lpad(round(value; digits=1), 6, "."))
end

**Ejercicio:**
Implementar la multiplicación matriz-vector $M \times V$ donde $M$ es una matriz y $V$ un vector, ambos con las dimensiones adecuadas. Envolver el cálculo en una función `multmatvec` y calcular los tiempos de ejecución con `@benchmark`.